In [1]:
import sys
    # caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, 'D:/books/python/0.   Fluent Python, 2nd Edition/example-code-2e/13-protocol-abc/')
sys.path

['D:\\books\\python\\0.   Fluent Python, 2nd Edition',
 'D:/books/python/0.   Fluent Python, 2nd Edition/example-code-2e/13-protocol-abc/',
 'C:\\Users\\lidan\\miniconda3\\python38.zip',
 'C:\\Users\\lidan\\miniconda3\\DLLs',
 'C:\\Users\\lidan\\miniconda3\\lib',
 'C:\\Users\\lidan\\miniconda3',
 '',
 'C:\\Users\\lidan\\AppData\\Roaming\\Python\\Python38\\site-packages',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\magic_impute-2.0.4-py3.8.egg',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\seqc-0.2.0-py3.8.egg',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\weasyprint-56.1-py3.8.egg',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\cairocffi-1.3.0-py3.8.egg',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\win32',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\Pythonwin']

In [2]:
class Vowels:
    def __getitem__(self, i):
        return 'AEIOU'[i]

v = Vowels()
v[0]

'A'

In [3]:
v[-1]

'U'

In [4]:
for c in v: print(c)

A
E
I
O
U


In [5]:
'E' in v

True

In [7]:
import collections
Card = collections.namedtuple('Card', ['rank', 'suit'])
class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                       for rank in self.ranks]
    def __len__(self):
        return len(self._cards)
    def __getitem__(self, position):
        return self._cards[position]

In [8]:
deck = FrenchDeck()
len(deck)

52

In [9]:
deck.suits

['spades', 'diamonds', 'clubs', 'hearts']

In [10]:
deck.ranks

['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']

In [11]:
deck[0]

Card(rank='2', suit='spades')

In [12]:
deck[-1]

Card(rank='A', suit='hearts')

In [13]:
from collections import namedtuple, abc

Card = namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value):  # <1>
        self._cards[position] = value

    def __delitem__(self, position):  # <2>
        del self._cards[position]

    def insert(self, position, value):  # <3>
        self._cards.insert(position, value)


In [14]:
deck = FrenchDeck2()
len(deck)

52

In [15]:
deck[-1]

Card(rank='A', suit='hearts')

In [16]:
deck.ranks

['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']

In [17]:
from random import shuffle
l = list(range(10))
shuffle(l)
l

[2, 3, 6, 9, 4, 0, 5, 7, 1, 8]

In [19]:
from random import shuffle
from frenchdeck2 import FrenchDeck2
deck = FrenchDeck2()
shuffle(deck)

In [22]:
deck[:10]

[Card(rank='4', suit='hearts'),
 Card(rank='Q', suit='clubs'),
 Card(rank='4', suit='spades'),
 Card(rank='10', suit='hearts'),
 Card(rank='A', suit='clubs'),
 Card(rank='J', suit='diamonds'),
 Card(rank='2', suit='diamonds'),
 Card(rank='K', suit='clubs'),
 Card(rank='Q', suit='hearts'),
 Card(rank='7', suit='clubs')]

In [20]:
deck[-1]

Card(rank='A', suit='diamonds')

In [21]:
deck.ranks

['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']

In [3]:
# tag::TOMBOLA_ABC[]

import abc

class Tombola(abc.ABC):  # <1>

    @abc.abstractmethod
    def load(self, iterable):  # <2>
        """Add items from an iterable."""

    @abc.abstractmethod
    def pick(self):  # <3>
        """Remove item at random, returning it.

        This method should raise `LookupError` when the instance is empty.
        """

    def loaded(self):  # <4>
        """Return `True` if there's at least 1 item, `False` otherwise."""
        return bool(self.inspect())  # <5>

    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:  # <6>
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)  # <7>
        return tuple(items)


# end::TOMBOLA_ABC[]


In [4]:
from tombola import Tombola

class Fake(Tombola):
    def pick(self):
        return 13

Fake

__main__.Fake

In [5]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract methods load

In [6]:
# tag::TOMBOLA_BINGO[]

import random

from tombola import Tombola


class BingoCage(Tombola):  # <1>

    def __init__(self, items):
        self._randomizer = random.SystemRandom()  # <2>
        self._items = []
        self.load(items)  # <3>

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)  # <4>

    def pick(self):  # <5>
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):  # <6>
        self.pick()

# end::TOMBOLA_BINGO[]


In [7]:
help(random.SystemRandom)

Help on class SystemRandom in module random:

class SystemRandom(Random)
 |  SystemRandom(x=None)
 |  
 |  Alternate random number generator using sources provided
 |  by the operating system (such as /dev/urandom on Unix or
 |  CryptGenRandom on Windows).
 |  
 |   Not available on all systems (see os.urandom() for details).
 |  
 |  Method resolution order:
 |      SystemRandom
 |      Random
 |      _random.Random
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  getrandbits(self, k)
 |      getrandbits(k) -> x.  Generates an int with k random bits.
 |  
 |  getstate = _notimplemented(self, *args, **kwds)
 |  
 |  random(self)
 |      Get the next random number in the range [0.0, 1.0).
 |  
 |  seed(self, *args, **kwds)
 |      Stub method.  Not used for a system random number generator.
 |  
 |  setstate = _notimplemented(self, *args, **kwds)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Random:
 |  
 |  __g

In [8]:
# tag::LOTTERY_BLOWER[]

import random

from tombola import Tombola


class LottoBlower(Tombola):

    def __init__(self, iterable):
        self._balls = list(iterable)  # <1>

    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            position = random.randrange(len(self._balls))  # <2>
        except ValueError:
            raise LookupError('pick from empty LottoBlower')
        return self._balls.pop(position)  # <3>

    def loaded(self):  # <4>
        return bool(self._balls)

    def inspect(self):  # <5>
        return tuple(self._balls)


# end::LOTTERY_BLOWER[]


In [9]:
from random import randrange

from tombola import Tombola

@Tombola.register  # <1>
class TomboList(list):  # <2>

    def pick(self):
        if self:  # <3>
            position = randrange(len(self))
            return self.pop(position)  # <4>
        else:
            raise LookupError('pop from empty TomboList')

    load = list.extend  # <5>

    def loaded(self):
        return bool(self)  # <6>

    def inspect(self):
        return tuple(self)

# Tombola.register(TomboList)  # <7>


In [10]:
TomboList.__mro__

(__main__.TomboList, list, object)

In [4]:
def double(x):
    return x * 2

In [5]:
double(1.5)

3.0

In [6]:
double('A')

'AA'

In [7]:
double([10, 20, 30])

[10, 20, 30, 10, 20, 30]

In [8]:
from fractions import Fraction
double(Fraction(2, 5))

Fraction(4, 5)

In [9]:
"""
A two-dimensional vector class

    >>> v1 = Vector2d(3, 4)
    >>> print(v1.x, v1.y)
    3.0 4.0
    >>> x, y = v1
    >>> x, y
    (3.0, 4.0)
    >>> v1
    Vector2d(3.0, 4.0)
    >>> v1_clone = eval(repr(v1))
    >>> v1 == v1_clone
    True
    >>> print(v1)
    (3.0, 4.0)
    >>> octets = bytes(v1)
    >>> octets
    b'd\\x00\\x00\\x00\\x00\\x00\\x00\\x08@\\x00\\x00\\x00\\x00\\x00\\x00\\x10@'
    >>> abs(v1)
    5.0
    >>> bool(v1), bool(Vector2d(0, 0))
    (True, False)


Test of ``.frombytes()`` class method:

    >>> v1_clone = Vector2d.frombytes(bytes(v1))
    >>> v1_clone
    Vector2d(3.0, 4.0)
    >>> v1 == v1_clone
    True


Tests of ``format()`` with Cartesian coordinates:

    >>> format(v1)
    '(3.0, 4.0)'
    >>> format(v1, '.2f')
    '(3.00, 4.00)'
    >>> format(v1, '.3e')
    '(3.000e+00, 4.000e+00)'


Tests of the ``angle`` method::

    >>> Vector2d(0, 0).angle()
    0.0
    >>> Vector2d(1, 0).angle()
    0.0
    >>> epsilon = 10**-8
    >>> abs(Vector2d(0, 1).angle() - math.pi/2) < epsilon
    True
    >>> abs(Vector2d(1, 1).angle() - math.pi/4) < epsilon
    True


Tests of ``format()`` with polar coordinates:

    >>> format(Vector2d(1, 1), 'p')  # doctest:+ELLIPSIS
    '<1.414213..., 0.785398...>'
    >>> format(Vector2d(1, 1), '.3ep')
    '<1.414e+00, 7.854e-01>'
    >>> format(Vector2d(1, 1), '0.5fp')
    '<1.41421, 0.78540>'


Tests of ``x`` and ``y`` read-only properties:

    >>> v1.x, v1.y
    (3.0, 4.0)
    >>> v1.x = 123
    Traceback (most recent call last):
      ...
    AttributeError: can't set attribute


Tests of hashing:

    >>> v1 = Vector2d(3, 4)
    >>> v2 = Vector2d(3.1, 4.2)
    >>> hash(v1), hash(v2)
    (7, 384307168202284039)
    >>> len(set([v1, v2]))
    2

Converting to/from a ``complex``:
# tag::VECTOR2D_V4_DEMO[]
    >>> from typing import SupportsComplex
    >>> v3 = Vector2d(1.5, 2.5)
    >>> isinstance(v3, SupportsComplex)  # <1>
    True
    >>> complex(v3)  # <2>
    (1.5+2.5j)
    >>> Vector2d.fromcomplex(4+5j)  # <3>
    Vector2d(4.0, 5.0)

# end::VECTOR2D_V4_DEMO[]
"""

from array import array
import math

class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

# tag::VECTOR2D_V4_COMPLEX[]
    def __complex__(self):
        return complex(self.x, self.y)

    @classmethod
    def fromcomplex(cls, datum):
        return cls(datum.real, datum.imag)  # <1>
# end::VECTOR2D_V4_COMPLEX[]


In [10]:
v1 = Vector2d(3, 4)
print(v1.x, v1.y)

3.0 4.0


In [11]:
x, y = v1
x, y

(3.0, 4.0)

In [12]:
v1

Vector2d(3.0, 4.0)

In [14]:
repr(v1)

'Vector2d(3.0, 4.0)'

In [13]:
eval(repr(v1))

Vector2d(3.0, 4.0)

In [16]:
eval('v1')

Vector2d(3.0, 4.0)

In [17]:
v1_clone = eval(repr(v1))
v1 == v1_clone

True

In [18]:
print(v1)

(3.0, 4.0)


In [19]:
octets = bytes(v1)
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [20]:
ord('d')

100

In [22]:
bytes([ord('d')])

b'd'

In [23]:
array('d', v1)

array('d', [3.0, 4.0])

In [24]:
bytes(array('d', v1))

b'\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [28]:
ord('a')

97

In [45]:
print(b'\x08')

b'\x08'


In [50]:
int.from_bytes(b'\x08@', "big")

2112

In [78]:
int.from_bytes(b'\x10@', "big")

4160

In [54]:
int.from_bytes(b'\x08', "big")

8

In [55]:
int.from_bytes(b'@', "big")

64

In [57]:
bin(64)

'0b1000000'

In [60]:
int('0b100001000000', 2)

2112

In [68]:
f'{0x08:0>8b}'

'00001000'

In [79]:
f'{0x10:0>8b}'

'00010000'

In [80]:
int('0b1000001000000', 2)

4160

In [65]:
bin(b'\x08')

TypeError: 'bytes' object cannot be interpreted as an integer

In [56]:
2**7

128

In [70]:
int.from_bytes(b'\x08', 'big')

8

In [75]:
chr(64)

'@'

In [76]:
ord('@')

64

In [77]:
abs(v1)

5.0

In [81]:
Vector2d(1, 0).angle()

0.0

In [82]:
Vector2d(1, 1).angle()

0.7853981633974483

In [83]:
format(Vector2d(1, 1), 'p')  # doctest:+ELLIPSIS

'<1.4142135623730951, 0.7853981633974483>'

In [84]:
v1.x, v1.y

(3.0, 4.0)

In [85]:
v1.x = 123

AttributeError: can't set attribute

In [87]:
v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2)

In [88]:
hash(v1), hash(v2)

(7, 384307168202284039)

In [89]:
len(set([v1, v2]))

2

In [90]:
from typing import SupportsComplex
v3 = Vector2d(1.5, 2.5)
isinstance(v3, SupportsComplex)  # <1>

True

In [91]:
complex(v3)  # <2>

(1.5+2.5j)

In [92]:
Vector2d.fromcomplex(4+5j)  # <3>

Vector2d(4.0, 5.0)

In [98]:
datum = 4+5j
datum.real

4.0

In [99]:
datum.imag

5.0

In [100]:
from __future__ import annotations

"""
A two-dimensional vector class

    >>> v1 = Vector2d(3, 4)
    >>> print(v1.x, v1.y)
    3.0 4.0
    >>> x, y = v1
    >>> x, y
    (3.0, 4.0)
    >>> v1
    Vector2d(3.0, 4.0)
    >>> v1_clone = eval(repr(v1))
    >>> v1 == v1_clone
    True
    >>> print(v1)
    (3.0, 4.0)
    >>> octets = bytes(v1)
    >>> octets
    b'd\\x00\\x00\\x00\\x00\\x00\\x00\\x08@\\x00\\x00\\x00\\x00\\x00\\x00\\x10@'
    >>> abs(v1)
    5.0
    >>> bool(v1), bool(Vector2d(0, 0))
    (True, False)


Test of ``.frombytes()`` class method:

    >>> v1_clone = Vector2d.frombytes(bytes(v1))
    >>> v1_clone
    Vector2d(3.0, 4.0)
    >>> v1 == v1_clone
    True


Tests of ``format()`` with Cartesian coordinates:

    >>> format(v1)
    '(3.0, 4.0)'
    >>> format(v1, '.2f')
    '(3.00, 4.00)'
    >>> format(v1, '.3e')
    '(3.000e+00, 4.000e+00)'


Tests of the ``angle`` method::

    >>> Vector2d(0, 0).angle()
    0.0
    >>> Vector2d(1, 0).angle()
    0.0
    >>> epsilon = 10**-8
    >>> abs(Vector2d(0, 1).angle() - math.pi/2) < epsilon
    True
    >>> abs(Vector2d(1, 1).angle() - math.pi/4) < epsilon
    True


Tests of ``format()`` with polar coordinates:

    >>> format(Vector2d(1, 1), 'p')  # doctest:+ELLIPSIS
    '<1.414213..., 0.785398...>'
    >>> format(Vector2d(1, 1), '.3ep')
    '<1.414e+00, 7.854e-01>'
    >>> format(Vector2d(1, 1), '0.5fp')
    '<1.41421, 0.78540>'


Tests of ``x`` and ``y`` read-only properties:

    >>> v1.x, v1.y
    (3.0, 4.0)
    >>> v1.x = 123
    Traceback (most recent call last):
      ...
    AttributeError: can't set attribute


Tests of hashing:

    >>> v1 = Vector2d(3, 4)
    >>> v2 = Vector2d(3.1, 4.2)
    >>> hash(v1), hash(v2)
    (7, 384307168202284039)
    >>> len(set([v1, v2]))
    2

Converting to/from a ``complex``:

    >>> from typing import SupportsComplex
    >>> v3 = Vector2d(1.5, 2.5)
    >>> isinstance(v3, SupportsComplex)  # <1>
    True
    >>> complex(v3)  # <2>
    (1.5+2.5j)
    >>> Vector2d.fromcomplex(4+5j)  # <3>
    Vector2d(4.0, 5.0)
"""

from array import array
import math
from typing import SupportsComplex, Iterator

class Vector2d:
    typecode = 'd'

    def __init__(self, x, y) -> None:
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self) -> float:
        return self.__x

    @property
    def y(self) -> float:
        return self.__y

    def __iter__(self) -> Iterator[float]:
        return (i for i in (self.x, self.y))

    def __repr__(self) -> str:
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self) -> str:
        return str(tuple(self))

    def __bytes__(self) -> bytes:
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other) -> bool:
        return tuple(self) == tuple(other)

    def __hash__(self) -> int:
        return hash(self.x) ^ hash(self.y)

    def __bool__(self) -> bool:
        return bool(abs(self))

    def angle(self) -> float:
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec='') -> str:
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets) -> Vector2d:
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

# tag::VECTOR2D_V5_COMPLEX[]
    def __abs__(self) -> float:  # <1>
        return math.hypot(self.x, self.y)

    def __complex__(self) -> complex:  # <2>
        return complex(self.x, self.y)

    @classmethod
    def fromcomplex(cls, datum: SupportsComplex) -> Vector2d:  # <3>
        c = complex(datum)  # <4>
        return cls(c.real, c.imag)
# end::VECTOR2D_V5_COMPLEX[]


In [101]:
math.hypot(3, 4)

5.0

In [102]:
Vector2d.fromcomplex(4+5j)  # <3>

Vector2d(4.0, 5.0)

In [103]:
v2 = Vector2d(3,4)
v2

Vector2d(3.0, 4.0)

In [104]:
v2.fromcomplex(4+5j)  # <3>

Vector2d(4.0, 5.0)

In [105]:
from typing import Protocol, runtime_checkable, Any

@runtime_checkable
class RandomPicker(Protocol):
    def pick(self) -> Any: ...


In [107]:
import random
from typing import Any, Iterable, TYPE_CHECKING

#from randompick import RandomPicker  # <1>

class SimplePicker:  # <2>
    def __init__(self, items: Iterable) -> None:
        self._items = list(items)
        random.shuffle(self._items)

    def pick(self) -> Any:  # <3>
        return self._items.pop()

def test_isinstance() -> None:  # <4>
    popper: RandomPicker = SimplePicker([1])  # <5>
    assert isinstance(popper, RandomPicker)  # <6>

def test_item_type() -> None:  # <7>
    items = [1, 2]
    popper = SimplePicker(items)
    item = popper.pick()
    assert item in items
    if TYPE_CHECKING:
        reveal_type(item)  # <8>
    assert isinstance(item, int)


In [111]:
!mypy example-code-2e/13-protocol-abc/typing/randompick_test.py

example-code-2e\13-protocol-abc\typing\randompick_test.py:24: note: Revealed type is "Any"
Success: no issues found in 1 source file


In [110]:
3**27

7625597484987

In [109]:
27**3

19683

In [113]:
from typing import Protocol, runtime_checkable
#from randompick import RandomPicker

@runtime_checkable  # <1>
class LoadableRandomPicker(RandomPicker, Protocol):  # <2>
    def load(self, Iterable) -> None: ...  # <3>
